### Riddler Classic: 

From Oliver Ruebenacker comes a “classic” indeed:

You have four standard dice, and your goal is simple: Maximize the sum of your rolls. So you roll all four dice at once, hoping to achieve a high score.

But wait, there’s more! If you’re not happy with your roll, you can choose to reroll zero, one, two or three of the dice. In other words, you must “freeze” one or more dice and set them aside, never to be rerolled.

You repeat this process with the remaining dice — you roll them all and then freeze at least one. You repeat this process until all the dice are frozen.

If you play strategically, what score can you expect to achieve on average?

### Solution Thoughts:

Strategy is always a bit tough, my assumption to start is that we need to lean on likelihood to dictate how we make a decision. 

**Strategy**:
- For each roll we need to freeze at least one of our dice, so we will take the highest and freeze. 
- For each other dice we can determine likelihood of getting the same value or greater over the next `n` rolls. 
    - Depending on the above, we make a decision:
        - If `P(Greater roll) >= 0.5`, then don't freeze
        - Otherwise, freeze

In [1]:
def greaterVal(rolls, current, sides = 6):
    """
    After n rolls, what is likelihood we roll less than or equal to current?
    
    rolls: Remaining rolls 
    current: Current value of roll
    """
    pr = (current)/sides
    return 1 - (pr** rolls)

In [2]:
### Testing: 

### If we roll a 6, we shouldn't be able to roll anything higher:
assert(greaterVal(4,6) == 0)

### If we roll a 5 we can think of this as:
### Likelihood of rolling a 5 or less each time
### Or, likelihood of not rolling a six... 5/6 ** 3
### So likelihood of rolling at least one six is just 1 - (5/6) ** 3
assert(round(greaterVal(3, 5),3) == 0.421)

### If we roll a 3 we should see pretty high likelihood here
greaterVal(3,3)

0.875

In [3]:
from scipy import stats

# k is discrete event (typically X), n is total trials
X = stats.binom(3, 1/6)
1 - X.cdf(0)

0.42129629629629617

In [4]:
X = stats.binom(3, 1/6)
1 - X.cdf(0)

0.42129629629629617

### Single Game: 

- We start with 4 random rolls
- Our highest dice needs to be stored off 

In [5]:
from dataclasses import dataclass
import random

In [6]:
@dataclass(frozen=False) # we want immutability 
class die:
    n: int
        
    def roll(self):
        return [random.randint(1,6) for x in range(self.n)]
        
    def __str__(self):
        return f"Remaining dice: {self.n}"

In [7]:
test = die(4)
test.roll()

[2, 2, 5, 1]

In [8]:
n = 4
frozen = []
i = 0

#### entering rolls ####
while True:
    # build out starting list
    die_obj = die(n)
    i += 1

    # Roll all dice
    roll_out = die_obj.roll()
    
    # Debugging:
    print(f"On roll {i} we rolled {roll_out}")

    # Find max roll by sorting & popping last element
    roll_out.sort()
    frozen.append(roll_out.pop(-1))

    # Determine max remaining rolls (this might be incorrect given we won't reroll some)
    max_rolls = len(roll_out)

    # For each of the remaining die, determine if we max out rolls if likelihood of greater > 0.5
    probs = [greaterVal(max_rolls, die_val) > 0.5 for die_val in roll_out]

    # if any probs are True, then we will try rerolling those. 
    # if False, then we append to frozen list 
    combined = zip(probs, roll_out)
    n = 0 # reset n
    for p, roll in combined:
        if p:
            n += 1
        else:
            frozen.append(roll)
            
    # Debugging:
    print(f"After roll {i} we have frozen set of {frozen}")
    
    if n == 0:
        final_val = sum(frozen)
        print(f"Total score: {final_val}")
        break

On roll 1 we rolled [5, 2, 2, 3]
After roll 1 we have frozen set of [5]
On roll 2 we rolled [1, 3, 2]
After roll 2 we have frozen set of [5, 3]
On roll 3 we rolled [1, 1]
After roll 3 we have frozen set of [5, 3, 1]
On roll 4 we rolled [1]
After roll 4 we have frozen set of [5, 3, 1, 1]
Total score: 10


### Move Into Simulation: 

Doing a bit of clean-up

In [9]:
sim = 100
final_val = 0 

for _ in range(sim):
    n = 4
    frozen = []

    while True:
        # build out starting list
        die_obj = die(n)

        # Roll all dice
        roll_out = die_obj.roll()

        # Find max roll by sorting & popping last element
        roll_out.sort()
        frozen.append(roll_out.pop(-1))

        # Determine max remaining rolls (this might be incorrect given we won't reroll some)
        max_rolls = len(roll_out)

        # For each of the remaining die, determine if we max out rolls if likelihood of greater > 0.5
        probs = [greaterVal(max_rolls, die_val) > 0.5 for die_val in roll_out]

        # if any probs are True, then we will try rerolling those. 
        # if False, then we append to frozen list 
        combined = zip(probs, roll_out)
        n = 0 # reset n
        for p, roll in combined:
            if p:
                n += 1
            else:
                frozen.append(roll)

        if n == 0:
            final_val+= sum(frozen)
            break

print(f"For sim of {sim}: {final_val / sim:.3f}")

For sim of 100: 18.390


### Cleaner Simulation - Larger Ranges

In [13]:
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000, 2_000_000]

for sim in sim_list:
    final_val = 0 

    for _ in range(sim):
        n = 4
        frozen = []

        while True:
            # build out starting list
            die_obj = die(n)

            # Roll all dice
            roll_out = die_obj.roll()

            # Find max roll by sorting & popping last element
            roll_out.sort()
            frozen.append(roll_out.pop(-1))

            # Determine max remaining rolls (this might be incorrect given we won't reroll some)
            max_rolls = len(roll_out)

            # For each of the remaining die, determine if we max out rolls if likelihood of greater > 0.5
            probs = [greaterVal(max_rolls, die_val) > 0.5 for die_val in roll_out]

            # if any probs are True, then we will try rerolling those. 
            # if False, then we append to frozen list 
            combined = zip(probs, roll_out)
            n = 0 # reset n
            for p, roll in combined:
                if p:
                    n += 1
                else:
                    frozen.append(roll)

            if n == 0:
                final_val+= sum(frozen)
                break

    print(f"For sim of {sim}: {final_val / sim:.3f}")

For sim of 100: 19.100
For sim of 1000: 18.637
For sim of 10000: 18.691
For sim of 100000: 18.712
For sim of 1000000: 18.701
For sim of 2000000: 18.701


### Riddler Classic: 

Extra credit: Instead of four dice, what if you start with five dice? What if you start with six dice? What if you start with N dice?

#### N = 5

In [14]:
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000, 2_000_000]

for sim in sim_list:
    final_val = 0 

    for _ in range(sim):
        n = 5
        frozen = []

        while True:
            # build out starting list
            die_obj = die(n)

            # Roll all dice
            roll_out = die_obj.roll()

            # Find max roll by sorting & popping last element
            roll_out.sort()
            frozen.append(roll_out.pop(-1))

            # Determine max remaining rolls (this might be incorrect given we won't reroll some)
            max_rolls = len(roll_out)

            # For each of the remaining die, determine if we max out rolls if likelihood of greater > 0.5
            probs = [greaterVal(max_rolls, die_val) > 0.5 for die_val in roll_out]

            # if any probs are True, then we will try rerolling those. 
            # if False, then we append to frozen list 
            combined = zip(probs, roll_out)
            n = 0 # reset n
            for p, roll in combined:
                if p:
                    n += 1
                else:
                    frozen.append(roll)

            if n == 0:
                final_val+= sum(frozen)
                break

    print(f"For sim of {sim}: {final_val / sim:.3f}")

For sim of 100: 24.590
For sim of 1000: 24.215
For sim of 10000: 24.277
For sim of 100000: 24.289
For sim of 1000000: 24.285
For sim of 2000000: 24.289


#### Iterate Through Various N

In [18]:
N_list = [n for n in range(4,15)]
sim = 100_000
for n_val in N_list:
    
    final_val = 0

    for _ in range(sim):

        frozen = []
        n = n_val
        
        while True:
            # build out starting list
            die_obj = die(n)

            # Roll all dice
            roll_out = die_obj.roll()

            # Find max roll by sorting & popping last element
            roll_out.sort()
            frozen.append(roll_out.pop(-1))

            # Determine max remaining rolls (this might be incorrect given we won't reroll some)
            max_rolls = len(roll_out)

            # For each of the remaining die, determine if we max out rolls if likelihood of greater > 0.5
            probs = [greaterVal(max_rolls, die_val) > 0.5 for die_val in roll_out]

            # if any probs are True, then we will try rerolling those. 
            # if False, then we append to frozen list 
            combined = zip(probs, roll_out)
            n = 0 # reset n
            for p, roll in combined:
                if p:
                    n += 1
                else:
                    frozen.append(roll)

            if n == 0:
                final_val+= sum(frozen)
                break

    print(f"For N = {n_val}: {final_val / sim:.3f}")

For N = 4: 18.716
For N = 5: 24.284
For N = 6: 30.003
For N = 7: 35.809
For N = 8: 41.660
For N = 9: 47.550
For N = 10: 53.480
For N = 11: 59.438
For N = 12: 65.377
For N = 13: 71.354
For N = 14: 77.342
